In [1]:
import pandas as pd
import numpy as np

In [53]:
fname = 'test-input.txt'
# fname = 'input.txt'

cave = np.array(pd.read_csv(fname, sep=',', header=None))
cave

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

In [153]:
def nearest_neighbors(a, i, j):
    x = [i, j]
    x_max = (a.shape[0]-1, a.shape[1]-1)

    neighbors = np.array([[x[0]-1, x[1]]])
    neighbors = np.append(neighbors, [[x[0]+1, x[1]]], axis=0)
    neighbors = np.append(neighbors, [[x[0], x[1]-1]], axis=0)
    neighbors = np.append(neighbors, [[x[0], x[1]+1]], axis=0)

    # Fix over-boundray values
    neighbors[np.where(neighbors<0)]=0
    neighbors[np.where(neighbors[:,0]>x_max[0]), 0] = x_max[0]
    neighbors[np.where(neighbors[:,1]>x_max[0]), 1] = x_max[1]
    
    return neighbors

def low(a, i, j):
    low_sum = 0
    for n in nearest_neighbors(a, i, j):
        low_sum += 1 if a[i][j] < a[n[0],n[1]] else low_sum
        
    low_pt = True if low_sum == 0 else False
    return low_pt

low_points = []
diff = np.full(cave.shape, 0)
for i in range(cave.shape[0]):
    for j in range(cave.shape[1]):
        diff[i][j] = low(cave,i,j)
        if diff[i][j] == 1:
            low_points.append((i,j))
            
diff

array([[0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]])

In [47]:
    
def low(a,i,j):

    if i == 0:
        min = True if a[i+1][j] > a[i][j] else False
    elif i == (a.shape[0]-1):
        min = True if a[i-1][j] > a[i][j] else False
    else:
        min = True if ( (a[i-1][j] > a[i][j]) and 
                        (a[i+1][j] > a[i][j])     ) else False

    if min:
        if j == 0:
            min = True if a[i][j+1] > a[i][j] else False
        elif j == (a.shape[1]-1):
            min = True if a[i][j-1] > a[i][j] else False
        else:
            min = True if ( (a[i][j+1] > a[i][j]) and
                            (a[i][j-1] > a[i][j])     ) else False

    return min   

low_points = []
diff = np.full(cave.shape, 0)
for i in range(cave.shape[0]):
    for j in range(cave.shape[1]):
        diff[i][j] = low(cave,i,j)
        if diff[i][j] == 1:
            low_points.append((i,j))

risk = ((cave+1) * diff).sum()

print(risk)

522


In [48]:
def flow(x_max, i0, j0, m, n, basin, cave, new_points):
    """ i0, j0: starting coords
        m = element i=0, j=1
        n = increase/decrease +1|-1
    """
    x = [i0,j0]
    prev = cave[tuple(x)]
    basin[tuple(x)]=1
    if n >0:
        down = False if x[m] == x_max[m] else True
    elif n<0:
        down = False if x[m] == 0 else True
    while down:
        x[m] = x[m]+n
        curr = cave[tuple(x)]
        down = prev < curr 
        down = False if curr == 9 else down
        if down:
            basin[tuple(x)] = 1
            new_points.append(tuple(x))
        if n >0:
            down = False if x[m] == x_max[m] else down
        elif n<0:
            down = False if x[m] == 1 else down
        prev = curr

    return basin, new_points

In [49]:
# Do one low point
def calc_basin(cave, low_point):

    basin = np.full(cave.shape, 0)
    x_max = (cave.shape[0]-1, cave.shape[1]-1)
    i0 = low_point[0]
    j0 = low_point[1]
    basin[i0,j0]=1
    pts = [[i0,j0]]

#     print('basin:',(i0,j0))
    t = 0
    while len(pts)>0:
        t += 1
        i0, j0 = pts.pop(len(pts)-1)
        # find flow points from i0, j0
        basin, pts = flow(x_max, i0, j0, 0, +1, basin, cave, pts) # i++
#         print('\t',pts1)
        basin, pts = flow(x_max, i0, j0, 0, -1, basin, cave, pts) # i--
#         print('\t',pts2)
        basin, pts = flow(x_max, i0, j0, 1, +1, basin, cave, pts) # j++
#         print('\t',pts3)
        basin, pts = flow(x_max, i0, j0, 1, -1, basin, cave, pts) # i--
#         print('\t',pts4)
        
    return basin, t

basins = []
sums = []
itr = []
# for low_point in [low_points[2]]:
for low_point in low_points:
    b, t = calc_basin(cave, low_point)
    itr.append(t)
    basins.append(b.tolist())
    sums.append(b.sum())

# total = 1
# sums = sorted(sums)
# for i in [1,2,3]:
#     total *= sums.pop()
# print(total)

In [50]:
# display(cave)
# display(diff)
# display(low_points)
print(np.array(itr)-np.array(sums))
# display(basins)

[   0    0   64  540    8   37  158  794  934 1069  173  790  167 1419
   23   86   48    8 1785   43  191 1784  206  634 1429    0  375  953
  252    7 2260    0  296    0   13  207  259    0  252    0   45  259
  152    0  105  974  166 2278    0  463  602  319   13  822  186    4
  551  114    4  173  422  690   14   91    0    5 3020   26  877  582
  220 2402   18 2847 1102  807 1514   30   20    4    3   21  515 2219
   42  241  792   23   18 2853 1248   28  153    3  341 2153    1    0
   35  222   16  422    5  669   54   21  368    0    0 1092  202    0
   56  650   17    9 3188   20  100 3853 1662    6    0    6 2291    1
    4  257  159 1671  109    9    0  889 2619  247 1214   27  105  641
 1513  270    1 3292  147   96    0  397    0    9    0    1    0    3
  394   52    0   32    8  630 2435    8  567   18    2    1  610 1156
    0   39  974   49   77    0 1846  614  136    0   11    5   36  478
   10    0    2    0  921    0    4   31  201  731  291  287 1537 1681
  234 

In [55]:
x = [[0,1], [2,3]]
x[-1]

[2, 3]